In [ ]:
# user-friendly print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
from crystallus import pbc_all_distances
from pymatgen import Structure

In [ ]:
pbc_all_distances?

`pbc_all_distance` is the function of calculating all distances between each site. The output is an upper triangular matrix.
Let's use the `178_224.cif` structure to compare our implementation with pymatgen.

In [ ]:
struct = Structure.from_file('178_224.cif')
struct

Firstly, you can see that pymatgen will normalize coords automatically when reading structure from CIF file.

In [22]:
pymatgen_dis = struct.distance_matrix
crystallus_dis = pbc_all_distances(struct.lattice.matrix.tolist(), struct.frac_coords.tolist())
crystallus_dis = np.array(crystallus_dis)

pymatgen_dis
crystallus_dis

array([[0.        , 6.88172759, 6.88172759, ..., 4.90348518, 7.94268292,
        4.4593836 ],
       [6.88172759, 0.        , 6.88172759, ..., 7.94268292, 4.90348518,
        4.4593836 ],
       [6.88172759, 6.88172759, 0.        , ..., 4.4593836 , 4.4593836 ,
        7.94268292],
       ...,
       [4.90348518, 7.94268292, 4.4593836 , ..., 0.        , 5.67329127,
        6.30652077],
       [7.94268292, 4.90348518, 4.4593836 , ..., 5.67329127, 0.        ,
        6.30652077],
       [4.4593836 , 4.4593836 , 7.94268292, ..., 6.30652077, 6.30652077,
        0.        ]])

array([[0.        , 6.88172759, 6.88172759, ..., 4.90348518, 7.94268292,
        4.4593836 ],
       [0.        , 0.        , 6.88172759, ..., 7.94268292, 4.90348518,
        4.4593836 ],
       [0.        , 0.        , 0.        , ..., 4.4593836 , 4.4593836 ,
        7.94268292],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 5.67329127,
        6.30652077],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        6.30652077],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

Secondly, our result is similar to that calculated by pymatgen. The only difference is our result has not the lower triangular part.

In [58]:
np.where((crystallus_dis > 0) & (crystallus_dis < 1))

pymatgen_dis[32, 58]
crystallus_dis[32, 58]

(array([32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34,
        34, 35, 35, 35, 35, 35, 35, 56, 56, 57, 57, 58, 58, 59, 59, 60, 60,
        61, 61, 62, 62, 63, 63, 64, 64, 65, 65, 66, 66, 67, 67]),
 array([58, 62, 66, 70, 74, 78, 59, 60, 65, 71, 72, 77, 56, 61, 67, 68, 73,
        79, 57, 63, 64, 69, 75, 76, 73, 79, 75, 76, 74, 78, 72, 77, 71, 77,
        68, 79, 70, 78, 69, 76, 69, 75, 71, 72, 70, 74, 68, 73]))

0.6042181598031264

0.6042181598031264

Also, very short distances exist. For example, the [32, 58] is 0.604218

Last, let's check the algorithm still usable when the coords are not normalized. The unnormalized coords saved in `unnor_coords.csv` are copied from `178_224.cif`.

In [59]:
unnor_coords = np.genfromtxt(
    'unnor_coords.csv',
    delimiter='  ',
    dtype=None,
    encoding='utf-8'
)
unnor_coords[0]

('Al', 'Al0', 1, 0.25, 0.25, 0.25, 1)

In [61]:
unnor_dis = pbc_all_distances(struct.lattice.matrix.tolist(), unnor_coords[['f3', 'f4', 'f5']].tolist())
unnor_dis = np.array(unnor_dis)

wrong_dis

array([[0.        , 8.42836057, 6.88172759, ..., 2.35215889, 2.35215889,
        2.35215889],
       [0.        , 0.        , 4.86611624, ..., 6.07620169, 6.07620169,
        6.07620169],
       [0.        , 0.        , 0.        , ..., 5.14370506, 5.14370506,
        5.14370506],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 3.84105938,
        3.84105938],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        3.84105938],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [63]:
np.where((wrong_dis > 0) & (wrong_dis < 1))

wrong_dis[20, 46]

(array([20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22,
        22, 23, 23, 23, 23, 23, 23, 44, 44, 45, 45, 46, 46, 47, 47, 48, 48,
        49, 49, 50, 50, 51, 51, 52, 52, 53, 53, 54, 54, 55, 55]),
 array([46, 50, 54, 58, 62, 66, 47, 48, 53, 59, 60, 65, 44, 49, 55, 56, 61,
        67, 45, 51, 52, 57, 63, 64, 61, 67, 63, 64, 62, 66, 60, 65, 59, 65,
        56, 67, 58, 66, 57, 64, 57, 63, 59, 60, 58, 62, 56, 61]))

0.6042181598031264

You can even the coords are not normalized, the calculation still gives a correct result.